# Área de Dados

A melhoria do processamento dos computadores, o surgimento de tecnologias como computação em nuvem, machine e a possibilidade da coleta de dados de diversos equipamentos através da Internet das Coisas (_Internet of Things_, _IoT_) fez com que uma enorme quantidade de dados (_Big Data_) começasse a ser produzido.

Esses dados, quando refinados, analisados e interpretados, podem trazer poderosas descobertas que melhoram as tomadas de decisão, ajudando a construir estratégias de mercado/negócios e a diminuir custos.

Existem diversas possibilidades de atuação na área, como por exemplo, na coleta e limpeza dos dados, análises exploratórias desses dados, construção e implantação de modelos de aprendizado de máquina (_machine learning_), e interpretação dos resultados desses modelos.

Os profissionais da área de dados podem ser divididos nas seguintes profissões:
- Engenheiro de dados: responsável pela coleta e limpeza dos dados
- Analista de dados: pode realizar limpeza dos dados e também é responsável pela análise exploratória desses dados
- Engenheiro de _machine learning_: constrói e implanta os modelos
- Cientista de dados: pode realizar qualquer uma dessas etapas

Neste estudo analisaremos os dados coletados na pesquisa [State of Data Brazil](https://www.stateofdata.com.br/) do ano de 2022, que fornece um panorama sobre o mercado de trabalho na área de dados no Brasil. Essa pesquisa foi realizada pela **Data Hackers** e a **Bain & Company**, no período de 10 de outubro a 28 de novembro de 2022. O arquivo .csv com as respostas dos 4.271 participantes pode ser obtido na plataforma [Kaggle](https://www.kaggle.com/datasets/datahackers/state-of-data-2022).

# State of Data Brazil 2022

In [ ]:
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('State_of_data_2022.csv', sep = ',')
df.drop("('P0', 'id')", axis = 1, inplace=True)
df.info()

FileNotFoundError: ignored

A pesquisa é dividida em 8 partes, totalizando 352 colunas (contando com a coluna de id, identificada como P0):

Parte 1 - Dados demográfico
Parte 2 - Dados sobre carreira
Parte 3 - Desafios dos gestores de times de dados
Parte 4 - Conhecimentos na área de dados
Parte 5 - Objetivos na área de dados
Parte 6 - Conhecimentos em Engenharia de Dados/DE
Parte 7 - Conhecimentos em Análise de Dados/DA
Parte 8 - Conhecimentos em Ciências de Dados/DS

Algumas perguntas podem ter respostas multi-valoradas, então ocupando mais de uma coluna no dataset. Dessa forma, cada coluna é identificada como uma tupla (Pergunta,Alternativa escolhida), e as perguntas são divididas em Parte, Letra da pergunta e Número da opção escolhida:

    P3_a_1 = Parte 3, pergunta (a), opção (1)

A fim de garantir o anonimato dos respondentes, os realizadores da pesquisa retiraram respostas que possam mostrar outliers e que fariam ser possível o reconhecimento de quem respondeu.

A seguir são mostradas as perguntas mais gerais para cada parte do questionário.

In [ ]:
# Separando por partes
df_parte_1 = df.filter(like = 'P1', axis = 1)
df_parte_2 = df.filter(like = 'P2', axis = 1)
df_parte_3 = df.filter(like = 'P3', axis = 1)
df_parte_4 = df.filter(like = 'P4', axis = 1)
df_parte_5 = df.filter(like = 'P5', axis = 1)
df_parte_6 = df.filter(like = 'P6', axis = 1)
df_parte_7 = df.filter(like = 'P7', axis = 1)
df_parte_8 = df.filter(like = 'P8', axis = 1)

In [ ]:
def get_perguntas(df_parte: pd.DataFrame,parte:int) -> None:
    columns   = list(df_parte.columns)
    print(f'Parte {parte} - Total de colunas: {len(columns)}')
    count = 0
    for c in columns:
        aux            = c.split('(')[1].split(')')[0]
        identificador1 = aux.split(',')[0]
        if len(identificador1.split('_')) <= 2:
            print(aux)
            count += 1
    print(f'Total de perguntas gerais: {count}')

In [ ]:
get_perguntas(df_parte_1,1)

In [ ]:
get_perguntas(df_parte_2,2)

In [ ]:
get_perguntas(df_parte_3,3)

In [ ]:
get_perguntas(df_parte_4,4)

In [ ]:
get_perguntas(df_parte_5,5)

In [ ]:
get_perguntas(df_parte_6,6)

In [ ]:
get_perguntas(df_parte_7,7)

In [ ]:
get_perguntas(df_parte_8,8)

E a seguir é mostrada a contagem dos tipos de dados para cada parte do questionário

In [ ]:
def get_tipos(df_parte: pd.DataFrame,parte:str) -> pd.DataFrame:
    tipos    = df_parte.dtypes.values.tolist()
    df_tipos = pd.DataFrame(tipos,columns=[parte])
    df_aggr  = df_tipos.groupby([parte])[parte].count()
    return df_aggr

def get_df_tipos() -> pd.DataFrame:
    dict_columns = {x: f'P{x+1}' for x in range(8)}
    dfs_tipos = []
    dfs_tipos.append(get_tipos(df_parte_1,'P1'))
    dfs_tipos.append(get_tipos(df_parte_2,'P2'))
    dfs_tipos.append(get_tipos(df_parte_3,'P3'))
    dfs_tipos.append(get_tipos(df_parte_4,'P4'))
    dfs_tipos.append(get_tipos(df_parte_5,'P5'))
    dfs_tipos.append(get_tipos(df_parte_6,'P6'))
    dfs_tipos.append(get_tipos(df_parte_7,'P7'))
    dfs_tipos.append(get_tipos(df_parte_8,'P8'))
    df_tipos = pd.concat(dfs_tipos,axis=1,ignore_index=True)
    df_tipos.rename(columns=dict_columns,inplace=True)
    df_tipos = df_tipos.fillna(0)
    df_tipos = df_tipos.astype('int32',errors='ignore')
    df_tipos['Total'] = df_tipos.loc[df_tipos.index].sum(axis=1)
    return df_tipos

In [ ]:
df_tipos = get_df_tipos()
display(df_tipos)

# Estatísticas gerais da idade dos profissionais

A coluna ('P1_a ', 'Idade') é a única do dataset que possui valores contínuos. As outras colunas de tipo float64 apenas indicam se o respondente escolheu uma dada opção de resposta (1 se escolheu a resposta e 0 para se não a escolheu).

Tirando algumas estatísticas básicas temos que a idade digitada pelos respondentes varia de 18 a 54 anos, com média de 31 anos e mediana de 30 anos.



In [ ]:
df_idade = pd.DataFrame(df_parte_1["('P1_a ', 'Idade')"].describe())
df_idade.rename(columns={"('P1_a ', 'Idade')": "Idade"}, inplace=True)
display(df_idade)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,9))

sns.histplot(data=df_parte_1, x="('P1_a ', 'Idade')",ax=ax[0], kde=True);
ax[0].set_title('Histograma da idade dos profissionais')
ax[0].set_xlabel('Idade')
ax[0].set_ylabel('Número de ocorrências')

df_parte_1.boxplot(column=["('P1_a ', 'Idade')"], ax=ax[1], grid=False)
ax[1].set_title('Boxplot das idades dos profissionais')
ax[1].set_ylabel('Idade')

# Distribuição das idades dos profissionais por gênero e nível de carreira

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,9))
df_parte_1.boxplot(column=["('P1_a ', 'Idade')"], by="('P1_b ', 'Genero')", ax=ax[0], grid=False);
ax[0].set_title('Boxplot das idades dos profissionais por gênero')
ax[0].set_ylabel('Idade')
ax[0].set_xlabel('Gênero')

df.boxplot(column=["('P1_a ', 'Idade')"], by="('P2_g ', 'Nivel')", ax=ax[1], grid=False);
ax[1].set_title('Boxplot das idades dos profissionais por nível')
ax[1].set_ylabel('Idade')
ax[1].set_xlabel('Nível');

# Número de profissionais por Estado e Região do Brasil

In [ ]:
num_profissionais_nao_moram_brasil = df_parte_1[df_parte_1["('P1_g ', 'vive_no_brasil')"]==False].shape[0]
mascara_sem_estado                 = (df_parte_1["('P1_i ', 'Estado onde mora')"].isnull()) & (df_parte_1["('P1_g ', 'vive_no_brasil')"]==True)
num_profissionais_nao_responderam  = df_parte_1[mascara_sem_estado].shape[0]

print(f'Número de profissionais que não moram no Brasil: {num_profissionais_nao_moram_brasil}')
print(f'Número de profissionais que não tem Estado: {num_profissionais_nao_responderam}')

In [ ]:
df_profissionais_por_estado2 = df_parte_1.groupby("('P1_i_1 ', 'uf onde mora')")["('P1_i_1 ', 'uf onde mora')"].count().sort_values().to_frame()
df_profissionais_por_estado2.rename(columns={"('P1_i_1 ', 'uf onde mora')": "Contagem"},inplace=True)
df_profissionais_por_estado2.reset_index(inplace=True)

In [ ]:
shp_brasil = 'BR_UF_2022/BR_UF_2022.shp'
gdf        = gpd.read_file(shp_brasil)
merged_df  = pd.merge(left=gdf,right=df_profissionais_por_estado2,
                     how='left', left_on='SIGLA_UF', right_on="('P1_i_1 ', 'uf onde mora')")
merged_df.fillna(0,inplace=True)


In [ ]:
coluna = "Contagem"
vmin = merged_df[coluna].min()
vmax = merged_df[coluna].max()
cmap = 'YlGnBu'
fig,ax = plt.subplots(1,figsize=(12,9))
ax.axis('off')
merged_df.plot(column=coluna, ax=ax, edgecolor='black', linewidth=1, cmap=cmap)
ax.set_title('Profissionais por Estado do Brasil')
sm = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=vmin, vmax=vmax),
                           cmap=cmap)
sm._A = []
cbaxes = fig.add_axes([0.15,0.25,0.01,0.4])
cbar   = fig.colorbar(sm, cax=cbaxes)

In [ ]:
num_prof_AM                = df_profissionais_por_estado['Amazonas (AM)']
num_prof_PA                = df_profissionais_por_estado['Pará (PA)']
mascara_prof_norte_sem_est = (df_parte_1["('P1_i_2 ', 'Regiao onde mora')"] == 'Norte') & (df_parte_1["('P1_i ', 'Estado onde mora')"].isnull())
num_prof_norte_sem_estado  = df_parte_1[mascara_prof_norte_sem_est].shape[0]

print(f'Número de profissionais no Norte: {df_profissionais_regiao["Norte"]}')
print(f'Número de profissionais no Amazonas: {num_prof_AM}')
print(f'Número de profissionais no Pará: {num_prof_PA}')
print(f'Número de profissionais no Norte mas sem Estado: {num_prof_norte_sem_estado}')

O Estado de São Paulo possui a grande maioria de profissionais e outros dois Estados do Sudeste, Minas Gerais e Rio de Janeiro, ficam com o 2º e 3º lugares, explicando a região Sudeste como a que mais tem profissionais de dados no Brasil. Os Estados do Acre, Amapá, Roraima, Tocantins (Norte) e Mato Grosso (Centro-Oeste) não tiveram profissionais.

Porém, ao filtrar quais profissionais são da região Norte mas não possuem resposta em Estado, obtemos a quantidade de 15, que é exatamente a quantidade que encontramos anteriormente de profissionais que não tem Estado.


# Número de profissionais por Estado e Região do Brasil separados pelo gênero

In [ ]:
coluna_genero = "('P1_b ', 'Genero')"
coluna_estado = "('P1_i ', 'Estado onde mora')"
df_genero_por_estado = df_parte_1.groupby([coluna_estado,coluna_genero]).size()
df_genero_por_estado.unstack().sort_values('Masculino').plot(kind='barh',stacked=True, ylabel= 'Estado', xlabel="Número de profissionais",
                                    title='Gênero dos profissionais por Estado');

In [ ]:
def tipo_profissionais_por_regiao(df_parte: pd.DataFrame, tipo: str, coluna_tipo: str, campo_sort: str) -> None:
    coluna_regiao      = "('P1_i_2 ', 'Regiao onde mora')"
    df_tipo_por_regiao = df_parte_1.groupby([coluna_regiao,coluna_tipo]).size().unstack().sort_values(campo_sort)
    df_tipo_por_regiao.plot(kind='barh',stacked=True, ylabel= 'Região', xlabel="Número de profissionais",
                                    title=f'{tipo} por Região')
    return df_tipo_por_regiao

In [ ]:
_ = tipo_profissionais_por_regiao(df_parte_1, 'Gênero dos profissionais', "('P1_b ', 'Genero')", 'Masculino')

Em relação ao gênero, como visto anteriormente, temos uma alta predominância do gênero Masculino, o que também é visto para todos os Estados do Brasil.

Dividindo o gênero por regiões, podemos confirmar um maior número de profissionais do gênero Feminino no Estado do Sudeste, o que é explicado no gráfico por Estado, com SP, MG, RJ tendo a maior quantidade de profissionais desse gênero.


# Número de profissionais por Estado e Região do Brasil separados por Cor/raça/etnia

In [ ]:
df_etnia = tipo_profissionais_por_regiao(df_parte_1, 'Cor/raça/etnia dos profissionais', "('P1_c ', 'Cor/raca/etnia')", 'Branca')


In [ ]:
df_etnia.fillna(0,inplace=True)
total_etnia_regiao   = df_etnia.sum(axis=1)
df_etnia_porcentagem = df_etnia.divide(total_etnia_regiao,axis=0) * 100
display(df_etnia_porcentagem)

Em relação à cor, raça e etnia, já tinha sido mostrada a maior predominância de pessoas brancas. No entanto, nas regiões Nordeste e Norte há uma menor diferença na proporção entre profissionais brancos e pardos (~49% e ~43.5% para o Norte; ~46% e 41% para o Nordeste).

O Sul é a região com maior predominância de profissionais brancos, chegando a ~81% dos profissionais na região.

In [ ]:
df_pcd = tipo_profissionais_por_regiao(df_parte_1, 'Profissionais PCD', "('P1_d ', 'PCD')", 'Não')

In [ ]:
 df_pcd.fillna(0,inplace=True)
display(df_pcd)
total_pcd  = df_pcd['Sim'].sum()
df_pcd_sim = pd.DataFrame(100 * df_pcd['Sim']/total_pcd).sort_values(by='Sim',ascending=False)
display(df_pcd_sim)

Pelo questionário são muito pouco profissionais PCD na área de dados no Brasil, sendo o Sudeste a região que apresenta a maior proporção desses profissionais, com ~61,5%.

# Profissionais por área de formação

In [ ]:
df_area_formacao       = df_parte_1.groupby("('P1_m ', 'Área de Formação')")["('P1_m ', 'Área de Formação')"].count().sort_values()
indexes_area_formacao  = df_area_formacao.index
values_area_formacao   = df_area_formacao.values

plt.barh(indexes_area_formacao, values_area_formacao)
plt.xlabel('Número de profissionais')
plt.title('Profissionais por área de formação');

In [ ]:
dict_areas = {
  'Ciências Sociais':                                                       'Humanas',
  'Ciências Biológicas/ Farmácia/ Medicina/ Área da Saúde':                 'Biológicas',
  'Química / Física':                                                       'Exatas',
  'Marketing / Publicidade / Comunicação / Jornalismo':                     'Humanas',
  'Outra opção':                                                            'Outra opção',
  'Estatística/ Matemática / Matemática Computacional/ Ciências Atuariais': 'Exatas',
  'Economia/ Administração / Contabilidade / Finanças/ Negócios':           'Humanas',
  'Outras Engenharias':                                                     'Exatas',
  'Computação / Engenharia de Software / Sistemas de Informação/ TI':       'Exatas'
}

grandes_areas          = [dict_areas[x] for x in df_parte_1["('P1_m ', 'Área de Formação')"].to_list() if not pd.isna(x)]
df_grandes_areas       = pd.DataFrame(grandes_areas,columns=['Grande Área'])
df_grandes_areas_group = df_grandes_areas.groupby('Grande Área')['Grande Área'].count().sort_values()
df_grandes_areas_group = 100 * df_grandes_areas_group / df_grandes_areas_group.sum()
indexes_grandes_areas  = df_grandes_areas_group.index
values_grandes_areas   = df_grandes_areas_group.values

labels = df_grandes_areas_group.index
plt.pie(df_grandes_areas_group, labels = labels, autopct='%1.1f%%',colors=sns.color_palette('muted'))
plt.title('Porcentagem de profissionais por grande área de formação');

A maioria dos profissionais de dados se formaram em áreas relacionadas à computação e exatas. No entanto, é possível notar que vários profissionais formados em áreas relacionadas à humanas também têm atuado com dados, chegando a aproximadamente 20% dos respondentes do questionário.